In [2]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import pandas as pd

#==================================#
#           Data dari FIBA         #
#==================================#
#Sementara pake bola size 7
MASS = 0.600 #580--620
CIRCUMFERENCE = 0.760 #750--770
RADIUS = CIRCUMFERENCE/(2*np.pi)
AREA = np.pi*RADIUS*RADIUS
RHO = 1.204
GRAVITY = 9.80
DRAG = 0.54
COR_HOOP = 0.6
COR_RING = 0.65
COR_RINGPLATE = 0.6



def ode_system(t, state, a,b):
    x, y, z, Vx, Vy, Vz = state
    dxdt = Vx
    dydt = Vy
    dzdt = Vz
    dVxdt = b * Vx**2
    dVydt = b * Vy**2
    dVzdt = a + b * Vz**2
    return [dxdt, dydt, dzdt, dVxdt, dVydt, dVzdt]

def GetInitVel (Rotation,Elevation,Velocity):#Tested
    Rotation = np.deg2rad(Rotation)
    Elevation = np.deg2rad(Elevation)
    VX = Velocity*np.cos(Elevation)*np.cos(Rotation)
    VY = Velocity*np.cos(Elevation)*np.sin(Rotation)
    VZ = Velocity*np.sin(Elevation)
    return VX,VY,VZ

def ShooterFahmi (X,Y,Z,Elevation,Rotation): #Sudut sementara dalam degree dan X,Y,Z koordinat Robot. (Hanya Shooter)

    #Spesifikasi Shooter 
    CenterToShaftX = -0.175
    RollerDiameter = 0.240
    CenterToShaftZ = 0.050 
    ShooterLength = 0.350 
    # Vmaks = 12

    #Konversi Sudut
    ShaftAngle = np.deg2rad(90-Elevation)
    Elevation = np.deg2rad(Elevation)
    Rotation = np.deg2rad(Rotation)


    #Perhitungan Posisi Bola saat ditembak
    InitialX = X - (CenterToShaftX + ((ShooterLength-RollerDiameter)/2 + RollerDiameter)*np.cos(ShaftAngle))*np.cos(Rotation)
    InitialY = Y + (CenterToShaftX + ((ShooterLength-RollerDiameter)/2 + RollerDiameter)*np.cos(ShaftAngle))*np.sin(Rotation)
    InitialZ = Z + CenterToShaftZ + ((ShooterLength-RollerDiameter)/2 + RollerDiameter)*np.sin(Elevation)
    return InitialX,InitialY,InitialZ


def calculate_constant(mass,gravity , C_Drag, Area, rho):
    
    A = 9.81
    B = -(C_Drag * rho * Area) / (2 * mass)
    
    return A,B


#==============================#
#       Collision Event        #
#==============================#
def Groundhit (t,y,A,B):
    return y[2] - 0
Groundhit.terminal = True
Groundhit.direction = -1 #Trigger when decreasing


#===ROBOT POS====#

X0 = 0
Y0 = 0
Z0 = 0
Drag_Var_range = np.arange(0.5,0.6,0.01)

V_error_range = np.arange(-5,5,1)
hit_data = []
#===Velocity and Position Parameter
for Drag_var in Drag_Var_range:
   for V_error in V_error_range:
    

    Voutput = 10 + V_error
    Elevation = 40
    Rotation = 0



    SpeedVector = GetInitVel(Rotation,Elevation,Voutput) 
    Initialpos = ShooterFahmi(X0,Y0,Z0,Elevation,Rotation=0)
    InitialState = np.append(Initialpos,SpeedVector)

    timestart = 0
    timestop = 20
    solutiondensity = 1000
    A,B = calculate_constant(MASS,GRAVITY,Drag_var,AREA,RHO)
    # print(A,B)

    
    #Jumlah T_eval = TimeStop/SolutionDensity --> regardless berapa waktu sampai time stop
    Trajectory = solve_ivp(ode_system, [timestart,timestop], InitialState,args=(A,B), events=[Groundhit],t_eval=np.linspace(timestart,timestop,solutiondensity), dense_output=True, method='RK45'  )

    ImpactPoint = Trajectory.y[0,-1]

    hit_data.append([V_error,Drag_var,ImpactPoint])



#==========================#
#     Visualisasi Table    #
#==========================#
# Print data
print(f"{'V_error':<10} {'Drag_Coeff':<10} {'ImpactPoint':<10}")
for row in hit_data:
    V_error, Drag_var, ImpactPoint = row
    print(f"{V_error:<10} {Drag_var:<10.2f} {ImpactPoint:<10.3f}")


df = pd.DataFrame(hit_data, columns=["V Error", "Drag Coefficient", "Impact Point (x)"])

# Save to Excel
df.to_excel("simulation_results.xlsx", index=False, sheet_name="Drag vs Velocity")


V_error    Drag_Coeff ImpactPoint
-5         0.50       44.114    
-4         0.50       49.326    
-3         0.50       53.978    
-2         0.50       58.180    
-1         0.50       62.010    
0          0.50       65.529    
1          0.50       68.784    
2          0.50       71.813    
3          0.50       74.644    
4          0.50       77.307    
-5         0.51       43.788    
-4         0.51       48.933    
-3         0.51       53.521    
-2         0.51       57.662    
-1         0.51       61.436    
0          0.51       64.902    
1          0.51       68.106    
2          0.51       71.087    
3          0.51       73.873    
4          0.51       76.492    
-5         0.52       43.468    
-4         0.52       48.547    
-3         0.52       53.073    
-2         0.52       57.156    
-1         0.52       60.874    
0          0.52       64.289    
1          0.52       67.444    
2          0.52       70.379    
3          0.52       73.121    
4        